# Einsendeaufgabe 1: Numerische Genauigkeit und Gleichungssysteme (100 Punkte)
In dieser Aufgabe sollen Sie ein wenig mehr Erfahrung mit NumPy und numerischen Methoden gewinnen.  
Zur Erinnerung empfehle ich an dieser Stelle, die Definition der [IEEE-Fließkommazahlen](https://de.wikipedia.org/wiki/IEEE_754) zu wiederholen.  

## Addition von Zahlen (20 Punkte) 

Gegeben sei ein Array *array*, dass 100 mal die Zahl $10^{-16}$ enthält und einmal (als ersten Eintrag) die Zahl $1$. 

In [2]:
import numpy as np

array = np.concatenate(([1], np.full(100, 1e-16)))
print(array)

[1.e+00 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16 1.e-16
 1.e-16]


**Aufgabe:** Addieren Sie alle Werte in `array`:
- mit der Funktion `np.sum()`
- mit einer Schleife, die von **vorne nach hinten** über `array` iteriert
- mit einer Schleife, die von **hinten nach vorne** über `array` iteriert

_Points:_ 6

In [3]:
sum_numpy = np.sum(array)
print(f"{sum_numpy=}")

sum_forward = 0.0
for x in array:
    sum_forward += x
print(f"{sum_forward=}")

sum_reverse = 0.0
for x in reversed(array):
    sum_reverse += x
print(f"{sum_reverse=}")

sum_numpy=np.float64(1.0000000000000084)
sum_forward=np.float64(1.0)
sum_reverse=np.float64(1.00000000000001)


<!-- BEGIN QUESTION -->

**Aufgabe:** Erklären Sie die Ergebnisse. Wie werden die Zahlen 1 und 1e-17 im Computer dargestellt? (ausführliche Erläuterung erforderlich!)  

_Points:_ 12

<div style="background-color: #f5f5f5; color: #37474f;">

## Numerische Genauigkeit von verschiedenen Additionsverfahren

### Unterschiede in der Summierung

Bei der Berechnung der Summe mit unterschiedlichen Verfahren ergeben sich leicht unterschiedliche Ergebnisse:

```python
sum_numpy   = np.float64(1.0000000000000084)
sum_forward = np.float64(1.0)
sum_reverse = np.float64(1.00000000000001)
```
Diese Unterschiede entstehen durch **Rundungsfehler** in der **Gleitkommadarstellung**, die auf der begrenzten Anzahl an Bits basiert, mit der reale Zahlen intern im Computer dargestellt werden.

---

## Gleitkommazahlen im IEEE 754 Standard

Python und NumPy verwenden standardmäßig `float64` (IEEE 754 Double Precision), das wie folgt aufgebaut ist:

- **1 Bit** für das Vorzeichen  
- **11 Bits** für den Exponenten  
- **52 Bits** für die Mantisse  

Insgesamt also **64 Bit**. Die reale Zahl wird durch folgende Formel dargestellt:

```
Zahl = (-1)^s · 1.m · 2^(e - 1023)
```

Dabei ist:

- `s` das Vorzeichenbit  
- `m` die Mantisse (ohne führende 1)  
- `e` der gespeicherte Exponent mit **Bias** = 1023  

---

## Maschinengenauigkeit

Die kleinste Zahl, die zu `1.0` addiert werden kann, sodass sich das Ergebnis **numerisch unterscheidet**, nennt man **Maschinengenauigkeit** oder **machine epsilon** $\varepsilon_{\text{mach}}$:

```python
np.finfo(float).eps
# Ergebnis: 2.220446049250313e-16
```

Zahlen kleiner als `eps` „verpuffen“ bei der Addition zu größeren Zahlen wie `1.0`, da sie außerhalb der darstellbaren Genauigkeit der Mantisse liegen. Wenn zwei Gleitkommazahlen stark unterschiedlich groß sind, kann die kleinere Zahl beim Addieren „verschluckt“ werden – das nennt man Verlust der Signifikanz:

```python
print(1e-16 < np.finfo(float).eps)  # True
print(1.0 + 1e-16 == 1.0)           # True
```

---

## Warum liefert `sum_forward` ein anderes Ergebnis als `sum_reverse`?

Angenommen:

```python
array = np.array([1.0] + [1e-16] * 100)
```

- `sum_forward` addiert zuerst `1.0` und dann viele kleine `1e-16`. Diese einzelnen Summanden sind kleiner als `eps` und **gehen verloren**.
- `sum_reverse` addiert zuerst die kleinen `1e-16` zu einem größeren Zwischenwert (z. B. `1e-14`), der dann zu `1.0` addiert wird und sich **merklich auswirkt**.
- `sum_numpy` verwendet oft optimierte oder paarweise Additionen (`pairwise summation`), was ebenfalls zu einem etwas besseren Ergebnis führen kann.

Gleitkommazahlen sind **nicht assoziativ**, d. h.:

```
(a + b) + c ≠ a + (b + c)
```

Das bedeutet, dass die Reihenfolge der Addition das Ergebnis beeinflussen kann, besonders wenn sehr kleine Zahlen in der Summe enthalten sind.

---

## Darstellung der Zahl `1.0` im IEEE 754-Standard

#### Mathematische Form:

$
1.0 = 1.0 \cdot 2^0 \\
\Rightarrow \text{Echter Exponent} = 0 \\
\Rightarrow \text{Gespeicherter Exponent} = 0 + \text{Bias} = 1023 = 01111111111_2
$

#### Mantisse:
- Die Zahl `1.0` entspricht im Binärsystem: $ 1.000\ldots0_2 $
- Die **führende 1 vor dem Komma** wird im IEEE 754-Format **nicht gespeichert** (implizit angenommen)
- Alle 52 Bits der Mantisse (also der Nachkommastellen) sind daher **0**

#### IEEE 754-Darstellung (64 Bit – `double precision`):

```
s        e (11 Bit)             m (52 Bit)
0   01111111111   0000000000000000000000000000000000000000000000000000
```

#### Zusammenfassung:
- **Vorzeichenbit (`s`)**: `0` → Zahl ist positiv  
- **Exponent (`e`)**: 1023 → gespeichert als `01111111111`  
- **Mantisse (`m`)**: nur Nullen (weil keine Nachkommastellen vorhanden sind)  
- Daraus ergibt sich der dargestellte Wert:
  
$
(-1)^0 \cdot 1.0 \cdot 2^{0} = 1.0
$

---

## Darstellung der Zahl `1e-17` im IEEE 754-Standard

### Mathematische Zerlegung von `1e-17`

Wir suchen $m$ und $e$, sodass:

$
1e-17 = 1.m \cdot 2^{e - 1023}
$

Um das umzuformen:

1. Schreibe `1e-17` als Zweierpotenz:
   $
   1e-17 = 10^{-17} \approx 2^{-56.14}
   $

2. Runde auf:
   $
   1e-17 \approx 2^{-56}
   $

3. Daraus folgt:
   $
   1e-17 = 1.0 \cdot 2^{-56}
   \Rightarrow e_{\text{effektiv}} = -56
   \Rightarrow \text{gespeicherter Exponent} = -56 + 1023 = 967
   $

- **Vorzeichenbit ($s$)**: `0` (positiv)  
- **Exponent ($e$)**: $967$ → binär: `01111001111`  
- **Mantisse ($m$)**: Nachkommabits von `1.0` (also alles Nullen)  

Die IEEE-754-Bitfolge sieht dann so aus (in 64 Bit):
```
s        e (11 Bit)             m (52 Bit)
0   01111001111   0000000000000000000000000000000000000000000000000000
```

`1e-17` ist **kleiner** als der kleinste darstellbare Unterschied zu `1.0` im IEEE 754-Standard (double precision), nämlich:

$
\varepsilon_{\text{mach}} \approx 2^{-52} \approx 2.22 \cdot 10^{-16}
$

---

### Fazit:

- Die Zahl `1.0` kann exakt im Format IEE 754 `float64` dargestellt werden.  
- Die Zahl `1e-17` ist **zu klein** und hat eine **unendliche Binärdarstellung**, was zu einer _Rundung_ führen muss. Eine präzise Darstellung innerhalb der 52 Bits der Mantisse ist nicht möglich. 


---

## Ab wann ist eine Summe nicht mehr darstellbar?

Eine einfache Faustregel:

> Zwei Zahlen `x` und `y` können exakt addiert werden, wenn  
> `| log2(x) - log2(y) | < 52` (bei `float64`)

Test in Python:

```python
def mindest_mantissenbits(x, y):
   return abs(np.log2(x) - np.log2(y))

mindest_mantissenbits(1, 1e-16) < 52  # False → nicht exakt addierbar
```

<div>

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

**Aufgabe:** Welches Verfahren verwendet `np.sum()` für die Addition?

_Points:_ 2

<div style="background-color: #f5f5f5; color: #37474f;">


## Kaskadensummation

Paarweise Summation (auch Kaskadensummation genannt) ist ein numerisch besseres Verfahren, bei dem die Werte in mehreren Schritten summiert werden. Dies bedeutet, dass Werte mit ähnlichen Größen zuerst zusammengefasst werden. Dadurch wird der Fehler reduziert, der entstehen kann, wenn immer kleinere Zahlen zu einem sehr großen Wert hinzugefügt werden (wie es bei der sequentiellen Summation der Fall wäre).

### Verfahren von `np.sum()`

NumPy verwendet für die Addition von Arrays ein Verfahren namens **Kaskadensummation** oder **paarweise Summation**. Dieses Verfahren reduziert Rundungsfehler im Vergleich zur sequentiellen Summation und führt in vielen Fällen zu einer höheren Genauigkeit. `np.sum()` kann interne Optimierungen verwenden, wie z. B. parallelisierte Summation, abhängig von der Größe des Arrays und der Hardware. In modernen Implementierungen von NumPy wird oft die `np.add.reduce()` Funktion verwendet, um die Summation durchzuführen, die im Hintergrund mit dem `ufunc`-Mechanismus arbeitet. Diese `reduce`-Methode summiert die Werte sequenziell, aber auch sie optimiert den Prozess, indem sie die Reihenfolge der Summanden variiert oder bei Bedarf parallelisiert.


Mehr Informationen zu dieser Technik gibt es auf [Wikipedia](https://en.wikipedia.org/wiki/Pairwise_summation) und in der [NumPy-Dokumentation](https://numpy.org/doc/stable/reference/generated/numpy.sum.html).

</div>

<!-- END QUESTION -->

## Grenzwerte von Funktionen (20 Punkte)

Geben sei die Funktion $f(x)=\frac{e^x-1}{x}$

Es gilt: $lim_{x \to 0}f(x) = 1$

**Aufgabe:** Schreiben Sie eine Python Funktion `f(x)`, mit deren Hilfe Sie $f(x)$ für $x= 10^{-1}, 10^{-2}, \cdots , 10^{-15}$ mit NumPy berechnen und geben Sie das Ergebnis aus.

**Aufgabe:** Berechnen Sie nun die Funktion für  $x= 10^{-16}, 10^{-17}, \cdots , 10^{-20}$

_Points:_ 8

<div style="background-color: #f5f5f5; color: #37474f;">

<h2>Berechnung der Funktion f(x) = (exp(x) - 1) / x </h2>

In dieser Aufgabe soll die Funktion f(x) für x = 10<sup>-1</sup>, 10<sup>-2</sup>, ..., 10<sup>-15</sup> berechnet werden. Danach soll die Funktion für noch kleinere Werte von x, nämlich x = 10<sup>-16</sup>, 10<sup>-17</sup>, ..., 10<sup>-20</sup>, berechnet werden. 

Der Grenzwert der Funktion für x → 0 ist 1. Dabei sollen mögliche numerische Probleme bei sehr kleinen x beobachtet werden.

<div>

In [ ]:
import numpy as np

# Definition der Funktion f(x) = (e^x - 1)/x
def f(x):
        return (np.exp(x) - 1) / x

# Bereich 1: x = 10^(-1) bis 10^(-15)
# Erzeuge die Werte x = 10^(-1), 10^(-2), ..., 10^(-15)
input_numbers = np.array([10**(-i) for i in range(1, 16)])
result_limits = f(input_numbers)

#Ausgabe
for x_val, fx in zip(input_numbers, result_limits):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-01, f(x) = 1.0517091807564771
x = 1e-02, f(x) = 1.005016708416795
x = 1e-03, f(x) = 1.0005001667083846
x = 1e-04, f(x) = 1.000050001667141
x = 1e-05, f(x) = 1.000005000006965
x = 1e-06, f(x) = 1.0000004999621837
x = 1e-07, f(x) = 1.0000000494336803
x = 1e-08, f(x) = 0.999999993922529
x = 1e-09, f(x) = 1.000000082740371
x = 1e-10, f(x) = 1.000000082740371
x = 1e-11, f(x) = 1.000000082740371
x = 1e-12, f(x) = 1.000088900582341
x = 1e-13, f(x) = 0.9992007221626409
x = 1e-14, f(x) = 0.9992007221626409
x = 1e-15, f(x) = 1.1102230246251565


In [ ]:
# Bereich 2: x = 10^(-16) bis 10^(-20)
input_numbers_2 = np.array([10**(-i) for i in range(16, 21)])
result_limits_2 = f(input_numbers_2)

#Ausgabe
for x_val, fx in zip(input_numbers_2, result_limits_2):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-16, f(x) = 0.0
x = 1e-17, f(x) = 0.0
x = 1e-18, f(x) = 0.0
x = 1e-19, f(x) = 0.0
x = 1e-20, f(x) = 0.0


<!-- BEGIN QUESTION -->

**Aufgabe:** Was fällt auf? Welche Erklärung haben Sie für das Ergebnis? (Ausführliche Erklärung erforderlich) 

_Points:_ 12

<div style="background-color: #f5f5f5; color: #37474f;">

<h2> Erklärung des Verhaltens der Funktion bei kleinen x-Werten </h2>

Für Werte von x zwischen `1e-01` bis `1e-14` liefert die Funktion erwartungsgemäß Ergebnisse nahe bei **1**. Ab etwa 1e-15 treten jedoch drastische Änderungen auf, die durch Rundungsfehler bei der Berechnung von `e^x - 1` bedingt sind. Besonders bei sehr kleinen x-Werten wird der Unterschied zwischen `e^x` und `1` auf dem Computer nicht mehr erkennbar, was zu fehlerhaften Ergebnissen führt. Eine stabilere Berechnung kann durch die Verwendung der Funktion `np.expm1(x)` von NumPy erreicht werden.


Von `1e-01` bis `1e-14` ergibt die Funktion erwartungsgemäß Werte, die sehr nahe bei **1** liegen.  
Doch ab etwa `1e-15` ändert sich das Verhalten drastisch:

x = 1e-15, f(x) = 1.1102230246251565   
x = 1e-16, f(x) = 0.0  
x = 1e-17, f(x) = 0.0  
x = 1e-18, f(x) = 0.0  
x = 1e-19, f(x) = 0.0  
x = 1e-20, f(x) = 0.0 

### Ergebnisbeobachtung

Ab etwa $x \approx 10^{-16}$ treten numerische Rundungsfehler sichtbar auf, abhängig von der Maschinengenauigkeit (`float64`).

<h3> Erklärung: Rundungsfehler </h3>

Für sehr kleine `x` gilt näherungsweise:  
`exp(x) ≈ 1 + x`

Wenn `x` so klein wird, dass `e^x` und `1` auf dem Computer **nicht mehr unterscheidbar** sind, wird der Ausdruck `e^x - 1` **numerisch ungenau** (_null_), da beide Terme sehr nah beieinander liegen → Verlust signifikanter Stellen.

> Beispiel:  
> `x = 1e-17` → `exp(x) ≈ 1.00000000000000001`  
> Die Differenz `e^x - 1` ist kleiner als die **Maschinengenauigkeit** `ε` (Epsilon).  
> Deshalb wird `e^x - 1` vom Computer als **0.0** berechnet, da ein 64-Bit-Gleitkommawert (float64) **nicht mehr ausreichend genau** ist.

<h3> Lösung: Stabilere Berechnung mit NumPy </h3>

NumPy bietet mit `np.expm1(x)` eine spezielle Funktion an, die den Ausdruck `exp(x) - 1` **numerisch stabiler** berechnet – insbesondere für sehr kleine `x`.

<div>

In [3]:
def f_stabil(x):
    return np.expm1(x) / x

input_numbers_3 = np.array([10**(-i) for i in range(10, 20)])
result_limits_3 = f_stabil(input_numbers_3)

#Ausgabe
for x_val, fx in zip(input_numbers_3, result_limits_3):
# Die Anwendung von zip gibt einen Iterator zurück, der in der Lage ist Tupel zu erzeugen
    print(f"x = {x_val:.0e}, f(x) = {fx}")

x = 1e-10, f(x) = 1.00000000005
x = 1e-11, f(x) = 1.000000000005
x = 1e-12, f(x) = 1.0000000000005
x = 1e-13, f(x) = 1.00000000000005
x = 1e-14, f(x) = 1.000000000000005
x = 1e-15, f(x) = 1.0000000000000007
x = 1e-16, f(x) = 1.0
x = 1e-17, f(x) = 1.0
x = 1e-18, f(x) = 1.0
x = 1e-19, f(x) = 1.0


<div style="background-color: #f5f5f5; color: #37474f;">

## Detaillierte Erklärung der Gleitkommadarstellung und Rundungsfehler

Die Präzision einer Gleitpunktzahl hängt von der Mantisse ab. Diese bildet neben dem Exponenten, der Basis und dem Vorzeichen die Gleitpunktzahl.

Solange man nicht explizit davon abweicht, wird `float64` in Python und NumPy verwendet für Gleitpunktzahlen.

- Eine `float32`-Mantisse besteht aus 23 Bit und eine `float64`-Mantisse aus 52 Bit (bei 64 Bit bleiben noch 11 Bit für den Exponenten und 1 Bit für das Vorzeichen).

Werden Werte, die mehr Bits zur Darstellung benötigen als die Mantisse zur Verfügung stellt, kommt es zu Rundungsfehlern, wie in den folgenden Beispielen erklärt.

## Fazit zur Gleitkommadarstellung und Rundungsfehlern

Die Präzision von Gleitkommazahlen ist durch die Anzahl der Bits in der Mantisse begrenzt, was zu Rundungsfehlern führt, wenn Zahlen mit mehr erforderlichen Bits dargestellt werden müssen. Besonders bei sehr kleinen Werten kann dies zu Verlusten an Genauigkeit führen, die sich durch den Einsatz stabiler Berechnungsmethoden wie z. B. `np.expm1(x)` verhindern lassen.

</div>

In [ ]:

def mindest_mantissenbits(x, y):
    # Berechnet die Anzahl der benötigten Mantissenbits für eine korrekte Berechnung
    # float128 um exaktere Praezision zu ermoeglichen
    return np.abs(np.log2(np.float128(x)) - np.log2(np.float128(y)))

# Beispiel zur Darstellbarkeit float32
# Mantisse = 1 wenn dargestellte Zahl Potenz von 2, also darstellbar
x = 1
a = np.float128(2**24-1)
x+a, np.float32(x+a), "darstellbar?:", mindest_mantissenbits(x,a) < 23 + 1
# Ergebnis: (16777216.0, 16777216.0, 'darstellbar?:', True)

# Beispiel zur Darstellbarkeit float32
# Dargestellte Zahl muss hier mit Mantisse von Mindestlaenge 24 dargestellt werden, nicht moeglich bei float32
b = np.float128(2**24)
x+b, np.float64(x+b), "darstellbar?:", mindest_mantissenbits(x,b) < 23 + 1
# Ergebnis: (16777217.0, 16777216.0, 'darstellbar?:', False)

# Diese Beispiele verdeutlichen, wie die Genauigkeit und die Darstellung von Zahlen durch die verwendete Gleitkommadarstellung beeinflusst werden.

# Weitere Beispiele zur Rundungsfehlerdarstellung
# In den folgenden Beispielen wird die Auswirkung von kleinen Zahlen in Gleitkommadarstellungen demonstriert:

# Beispiel 1 + 1e-16, vorkommend bei Summierung in Schleife von vorne
c = np.float128(1e-16)
x + c, np.float64(x + c), "darstellbar?:", mindest_mantissenbits(x, c) < 52 + 1
# Ergebnis: (1.0000000000000001, 1.0, 'darstellbar?:', False)

# Beispiel 1 + 1e-16 * 100, vorkommend bei Summierung in Schleife von hinten
d = np.float128(1e-16 * 100)
x + d, np.float64(x + d), "darstellbar?:", mindest_mantissenbits(x, d) < 52 + 1
# Ergebnis: (1.00000000000001, 1.00000000000001, 'darstellbar?:', True)